# Dependencies

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:

import datasets_handler
import zeroberto
import evaluation_metrics
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from zeroberto import ZeroBERTo
import pandas as pd
import numpy as np


# Dataset

In [3]:
which_dataset = 'ag_news'
# which_dataset = 'folhauol'
sample_size = 50000
hypothesis = "this text is about {}."
# hypothesis = "{}."


raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
raw_data[data_col] = raw_data[data_col].dropna()
classes_list = raw_data[class_col].drop_duplicates().to_list()
raw_data = evaluation_metrics.Encoder(raw_data,[class_col])

Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
train_dataset = raw_data.sample(sample_size,random_state=422).sort_index()
X = train_dataset[data_col].to_list()
Y_class =  train_dataset[class_col].to_list()
Y_code =  train_dataset[class_col+"_code"].to_list()
pd.Series(zip(Y_class,Y_code)).drop_duplicates()

0                    (business, 0)
28     (science and technology, 1)
158                    (sports, 2)
180                     (world, 3)
dtype: object

In [5]:
classes_list_encoded = evaluation_metrics.Encoder(pd.DataFrame({"classes":classes_list}))['classes_code']

# Model

In [6]:

model = ZeroBERTo(classes_list=classes_list,hypothesis_template=hypothesis,
                  train_dataset=train_dataset)
model.clusterModel.fit(model.initial_centroids)

KMeans(init=array([[-0.00430593,  0.02161113,  0.01563421, ..., -0.03426863,
        -0.05354397,  0.0109923 ],
       [-0.01198703,  0.00461293,  0.04327798, ..., -0.03150452,
        -0.02404456, -0.00423621],
       [ 0.04327394, -0.02182552,  0.03044848, ...,  0.02151219,
        -0.00977532,  0.05579362],
       [ 0.0047062 , -0.02189007,  0.06357092, ..., -0.01765898,
        -0.00142346, -0.01379836]]),
       max_iter=600, n_clusters=4, n_init=1, random_state=422)

In [7]:
%%time
X_emb = model.encode(X)

CPU times: user 1h 4min 6s, sys: 7min 56s, total: 1h 12min 3s
Wall time: 28min 9s


In [8]:
y_pred = model.clusterModel.predict(X_emb)

In [9]:
try: print(all_metrics)
except: pass
all_metrics = evaluation_metrics.get_metrics(y_pred,Y_code)
print(all_metrics)

{'weighted': [{'accuracy': 0.5401}, {'precision': 0.5931074147000197}, {'recall': 0.5401}, {'f1': 0.52228011760948}], 'macro': [{'accuracy': 0.5401}, {'precision': 0.5925337959994214}, {'recall': 0.5409651846614242}, {'f1': 0.5224419859385745}]}


In [10]:
y_probs = model.clusterModel.transform(X_emb)

In [11]:
df_results = model.evaluateLabeling(y_probs)


top 1: {'accuracy': 0.5}
top 2: {'accuracy': 0.5}
top 3: {'accuracy': 0.5}
top 4: {'accuracy': 0.5625}
top 5: {'accuracy': 0.55}
top 6: {'accuracy': 0.5}
top 7: {'accuracy': 0.5}
top 8: {'accuracy': 0.5}
top 9: {'accuracy': 0.5277777777777778}
top 10: {'accuracy': 0.525}
top 11: {'accuracy': 0.5454545454545454}
top 12: {'accuracy': 0.5625}
top 13: {'accuracy': 0.5961538461538461}
top 14: {'accuracy': 0.6071428571428571}
top 15: {'accuracy': 0.6}
top 16: {'accuracy': 0.59375}
top 50000: {'accuracy': 0.5401}


,class,prediction,prediction_code,class_code,top_probability
4,business,world,3,0,1.308632
6,business,business,0,0,1.197235
10,business,world,3,0,1.342313
16,business,business,0,0,1.236719
18,business,business,0,0,1.354064
...,...,...,...,...,...
127588,science and technology,sports,2,1,1.217498
127592,world,business,0,3,1.264553
127593,sports,sports,2,2,1.324019
127595,world,business,0,3,1.390479
